In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    ffn_hyperparameter_search,
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

Baseline: FFN

In [7]:
num_epochs = 100
batch = 32
hidden_dim_sizes = [[32,32]]#[[32,32], [64,64], [128,128], [256,256]]
dropout_rates = [0.2] #[0.5, 0.2, 0.1]
learning_rates = [1e-4]#[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

We use the ffn_hyperparameter_search function which loops through the different hidden dimensions, dropout rates and learning rates to find the best model for the validation set. We evaluate the model on several seeds and average the performance over the seeds.

In [8]:
ffn_current, best_ffn_current, _, __ = ffn_hyperparameter_search( 
    num_epochs=num_epochs,
    x_data=sbert_embeddings,
    y_data=y_data,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    batch_size=batch,
    data_split_seed=123,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:35<00:00, 35.71s/it]


In [9]:
ffn_current

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id
0,focal,0.510010,0.492842,"[0.5861513687600645, 0.3995327102803738]",0.239724,0.612100,0.581485,"[0.6946778711484595, 0.4682926829268293]","[32, 32]",0.2,0.0001,0,2,False,0
0,focal,0.510963,0.493644,"[0.5872888173773129, 0.4]",0.235819,0.619217,0.585631,"[0.703601108033241, 0.46766169154228854]","[32, 32]",0.2,0.0001,1,2,False,0
0,focal,0.510010,0.488524,"[0.5933544303797469, 0.3836930455635491]",0.240224,0.622776,0.584783,"[0.7103825136612023, 0.4591836734693877]","[32, 32]",0.2,0.0001,12,2,False,0
0,focal,0.514776,0.494918,"[0.5950676213206046, 0.3947681331747919]",0.238547,0.615658,0.576949,"[0.7049180327868853, 0.4489795918367347]","[32, 32]",0.2,0.0001,123,2,False,0
0,focal,0.521449,0.505040,"[0.5951612903225807, 0.41491841491841486]",0.235836,0.619217,0.581851,"[0.7068493150684931, 0.45685279187817257]","[32, 32]",0.2,0.0001,1234,2,False,0


In [10]:
best_ffn_current

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold
0,focal,0.510010,0.492842,"[0.5861513687600645, 0.3995327102803738]",0.239724,0.612100,0.581485,"[0.6946778711484595, 0.4682926829268293]","[32, 32]",0.2,0.0001,0,2,False
0,focal,0.510963,0.493644,"[0.5872888173773129, 0.4]",0.235819,0.619217,0.585631,"[0.703601108033241, 0.46766169154228854]","[32, 32]",0.2,0.0001,1,2,False
0,focal,0.510010,0.488524,"[0.5933544303797469, 0.3836930455635491]",0.240224,0.622776,0.584783,"[0.7103825136612023, 0.4591836734693877]","[32, 32]",0.2,0.0001,12,2,False
0,focal,0.514776,0.494918,"[0.5950676213206046, 0.3947681331747919]",0.238547,0.615658,0.576949,"[0.7049180327868853, 0.4489795918367347]","[32, 32]",0.2,0.0001,123,2,False
0,focal,0.521449,0.505040,"[0.5951612903225807, 0.41491841491841486]",0.235836,0.619217,0.581851,"[0.7068493150684931, 0.45685279187817257]","[32, 32]",0.2,0.0001,1234,2,False


In [11]:
best_ffn_current["f1"].mean()

0.49499358320974396

In [12]:
np.stack(best_ffn_current["f1_scores"]).mean(axis=0)

array([0.59140471, 0.39858246])